# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [12]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
# import rezaware as reza
from utils.modules.etl.load import sparkDBwls as sdb
from utils.modules.etl.transform import sparkCleanNRich as scne
from wrangler.modules.assets.etl import dataPrep as prep
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
#     reza = importlib.reload(reza)
    sdb = importlib.reload(sdb)
    scne = importlib.reload(scne)
    prep = importlib.reload(prep)
#     stats= importlib.reload(stats)
    
__desc__ = "analyze crypto market capitalization time series data"
# clsSDB = sdb.SQLWorkLoads(desc=__desc__)
clsSCNR=scne.Transformer(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
clsPrep =prep.RateOfReturns(desc=__desc__)
# ''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional DATAPREP-libraries in ETL-package of ASSETS-module imported successfully!
sparkNoSQLwls Class initialization complete
dataPrep Class initialization complete

Class initialization and load complete!


## Read data from mcap_past
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [55]:
import pandas as pd
_fpath = os.path.join('/home/nuwan/workspace/rezaware/',
                      'wrangler/data/assets/etl/tmp/assets_20220720.csv')
_piv_asset_list = pd.read_csv(_fpath)['asset_name'].tolist()
# _piv_asset_list[0:100]

In [62]:
_from_date = '2022-07-20'
_to_date = '2022-07-31'
_value_limit=100000000
_attr_prefix = 'mcap'
_piv_cols = _piv_asset_list[3500:]

_kwargs = {
    "DBNAME" : "tip",
    "TABLENAME" : 'mcap_past',
    "PARTCOLNAME":'asset_name',
    "DATECOLNAME":"_".join([_attr_prefix,'date']),
    "VALUECOLNAME":"_".join([_attr_prefix,'value']),
    "AGGREGATE":'avg',
    "IMPUTESTRATEGY":'mean',
    "PIVCOLUMNS":_piv_cols,
}

# _query = "select * from warehouse.mcap_past wmp "+\
#         f"where wmp.mcap_date between '{_from_date}' and '{_to_date}' "+\
#         f"and wmp.mcap_value > 10000 "
_query =f"SELECT wmp.mcap_past_pk, wmp.uuid, wmp.asset_symbol, "+\
        f"wmp.{_kwargs['DATECOLNAME']}, wmp.{_kwargs['VALUECOLNAME']}, "+\
        f"wmp.{_kwargs['PARTCOLNAME']}, wmp.currency, "+\
        f"wmp.created_dt,wmp.created_by,wmp.created_proc "+\
        f"FROM warehouse.mcap_past wmp WHERE 1=1 "+\
        f"AND wmp.{_kwargs['DATECOLNAME']} between '{_from_date}' AND '{_to_date}' "+\
        f"AND wmp.mcap_value > {_value_limit} "+\
        f"AND wmp.deactivate_dt IS NULL"
# print(_query)
mcap_sdf = clsPrep.read_n_clean_mcap(query=_query,**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

Loaded 5083 rows and 10 columns


In [63]:
from pyspark.sql import functions as F
mcap_sdf=mcap_sdf.filter(F.col('asset_name').isin(_piv_cols))
print("rows = ",mcap_sdf.count())
print(mcap_sdf.show(n=1,vertical=True))

rows =  530


-RECORD 0----------------------------
 asset_name   | ake                  
 mcap_date    | 2022-07-20 00:00:00  
 mcap_past_pk | 511176               
 uuid         | 63945b519d51d3449... 
 asset_symbol | cake                 
 currency     | null                 
 created_dt   | 2023-02-14 17:47:... 
 created_by   | farmraider           
 created_proc | wrangler_assets_e... 
 mcap_value   | 508889928.3761090... 
only showing top 1 row

None


## Compute LogROR for all assets

In [66]:
from pyspark.sql import functions as F

_ror='SIMP'

if _ror in ['NATLOG','LOG2','LOG10','LN']:
    _ror_col="_".join([_attr_prefix,'log','ror'])
elif _ror=='SIMP':
    _ror_col="_".join([_attr_prefix,_ror.lower(),'ror'])
else:
    pass
_kwargs["PREVALCOLNAME"]="_".join([_attr_prefix,'lag'])
_kwargs["DIFFCOLNAME"] = "_".join([_attr_prefix,'diff'])
_kwargs["RORCOLNAME"] = _ror_col
_kwargs["PARTITIONS"] = 2
#     "PARTCOLNAME":'asset_name',
#     "DATECOLNAME":"_".join([_attr_prefix,'date']),
#     "VALUECOLNAME":"_".join([_attr_prefix,'value']),
#     "COLUMN":"_".join([_attr_prefix,'date']),
#     "FROMDATETIME":_from_date,
#     "TODATETIME":_to_date,

_ror_data, _ror_col = clsPrep.calc_ror(
    data=mcap_sdf,
    ror_type=_ror,
    num_col =_kwargs['VALUECOLNAME'],
    part_col=_kwargs['PARTCOLNAME'],
    date_col=_kwargs['DATECOLNAME'],
    **_kwargs,
)

_ror_data.filter(F.col(_kwargs["PARTCOLNAME"]).isNotNull()).show(n=1,vertical=True)

-RECORD 0-----------------------------
 asset_name    | ala                  
 mcap_date     | 2022-07-20 00:00:00  
 mcap_past_pk  | 641030               
 uuid          | 6392d6fa9d51d3449... 
 asset_symbol  | gala                 
 currency      | usd                  
 created_dt    | 2023-02-14 18:30:... 
 created_by    | farmraider           
 created_proc  | wrangler_assets_e... 
 mcap_value    | 445818989.2479350... 
 mcap_simp_ror | null                 
only showing top 1 row



## Write ROR data to DB

In [67]:
_upsert_sdf=_ror_data.select('*')\
    .filter(F.col(_kwargs["RORCOLNAME"]).isNotNull())

_records=clsPrep.write_data_to_db(
    data=_upsert_sdf,
#     **kwargs,
)
print("Upserted %d records" % _records)

Validating upsert attributes and parameters ...


Wait a moment, writing data to postgresql tip database ...


Upserted 492 records
